In [ ]:
#using Markdown; Markdown.parse_file("README.md")

In [ ]:
MODEL_PATH_KEY = :wNB;
const FLTYPE = Float32;
include("heavyloadings.jl");

In [ ]:
half_b = load(MODEL_FOLDER*"\\bunny\\half_bunny_unmod.ply");
half_b_rot = load(MODEL_FOLDER*"\\bunny\\half_bunny_rot1.ply");

In [ ]:
include("firsticp.jl")

In [ ]:
half_b_sc = scaleMesh(half_b,10);
half_b_rot_sc = scaleMesh(half_b_rot,10);

In [ ]:
IJuliaCell(vis)

In [ ]:
placeMeshInLife(vis,half_b_sc,"hbunny","darkred");
placeMeshInLife(vis,half_b_rot_sc,"rotbunny","olivedrab2")

## Old workflow
Every array is using `Point3f0`

```julia
#Workflow: (reading, reference)
#Input meshes/pointclouds
reference_pc = deepcopy(half_b_sc);
reading_pc = noisifyMesh(half_b_rot_sc);
#Arrays of Point3f0
reference_array = vertices(reference_pc);
reading_array = vertices(reading_pc);
#KD tree for the reference array
#kdtree_ref = KDTree(reference_array);

#Sampling random elements from the reference array
sampled_points_ref = randomSamplePoints(reference_array,0.5);
#Building KD tree for the sampled points
kdtree_sampled_ref = KDTree(sampled_points_ref);

#pairing the points in the two meshes
indexer,fl_traits = createKnnPairArray(reading_array,kdtree_sampled_ref,sampled_points_ref,true);
#rejecting the worst 20% based on the distance
rej_is,rej_fltrs = rejectWorstPercent(indexer,fl_traits,0.2);
```

## New workflow
Using Homogeneous vector instead of `Point3f0`
### Workflow lépései:
1. [x] reading és ref mesh-t homogén koordinátás SVector/MVector**(?)** tömbbé alakítani
1. [x] a teljes referencia tömbből kd-tree
### Iteráció
1. [x] reading tömbből sampling
1. [x] samplinged tömbhöz párkeresés a fából
1. ##### Worst `x`% rejection
  * [x] párok sorbarendezése távolság szerint
  * [x] a sorbarendezett párok `x`%-nak kidobása
1. ##### 2.5sigma rejection
  * [ ] szórás számítása
  * [ ] 2.5 sigmánál nagyobbak kidobása
1. [ ] A\b összeállítása és megoldása
1. [ ] transzformáció a megoldással
1. [ ] transzformáció mentése vizualizációhoz
1. [ ] kilépési feltétel check

### Iteráció vége

11. [ ] place in MeshCat
12. [ ] animate in MeshCat


In [ ]:
#Workflow: (reading, reference)
#Input meshes/pointclouds
reference_pc = deepcopy(half_b_sc);
reading_pc = noisifyMesh(half_b_rot_sc);
#Arrays of SVector
reference_array = convert2HomogeneousArray(vertices(reference_pc),FLTYPE);
reading_array = convert2HomogeneousArray(vertices(reading_pc),FLTYPE);
#Sampling random elements from the reference and the reading array
sampled_points_ref = randomSamplePoints(reference_array,50);
sampled_points_red = randomSamplePoints(reading_array,50);
#Building KD tree for the sampled ref points and the whole ref array
kdtree_sampled_ref = KDTree(sampled_points_ref);
kdtree_ref = KDTree(reference_array);

#Iterate from here

#pairing the points in the two meshes
indM, traitM = createKnnPairArray(sampled_points_red,kdtree_ref);
#sorting the indexes by the distances in the trait matrix
s_indM, s_it = sortIndexes(indM, traitM[:,2]);
#remove the worst 10%
ch_ind, ch_it = chopEndOfArray(20,s_indM,s_it);

Transformation matrix:

Tkkv * sampled_points_red[ch_ind[:,2]]= reference_array[ch_ind[:,1]]

after transpose 

sampled_points_red_transp * Tkkv_transp = reference_array_transp

this is a form of A*x = b

Solution for Tkkv_transp

x = A \ b

Tkkv_transp = sampled_points_red_transp \ reference_array_transp


In [ ]:
#make the transformation matrix

#for debug make a subarray
smpl_ch = sampled_points_red[ch_ind[:,2]];
ref_ch = reference_array[ch_ind[:,1]];

In [ ]:
#rtanspose them
smpl_tp = transpose(smpl_ch);
ref_tp = transpose(ref_ch);

In [15]:
Tkkv_transposed = smpl_tp\ref_tp

MethodError: MethodError: no method matching pinv(::Array{SArray{Tuple{4},Float32,1,4},1}, ::SArray{Tuple{4},Float32,1,4})
Closest candidates are:
  pinv(::AbstractArray{T,1}) where T at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\generic.jl:808
  pinv(::AbstractArray{T,1}, !Matched::Real) where T at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\generic.jl:808

In [16]:
Tkkv_transposed = smpl_ch\ref_ch

MethodError: MethodError: no method matching pinv(::Array{SArray{Tuple{4},Float32,1,4},1}, ::SArray{Tuple{4},Float32,1,4})
Closest candidates are:
  pinv(::AbstractArray{T,1}) where T at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\generic.jl:808
  pinv(::AbstractArray{T,1}, !Matched::Real) where T at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.0\LinearAlgebra\src\generic.jl:808